In [48]:
import cv2
import numpy as np
import os

input_path = "archive\\Ulcer\\Ulcer_2024-08-07-08-31-02_23691.bmp"
#input_path = "Ulcer_2024-08-07-08-25-26_23438.bmp"

image = cv2.imread(input_path)

## Szukanie i usuwanie obramowania

In [51]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#obszar różny od czarnego
mask = np.where(gray > 10, 255, 0).astype(np.uint8)  
# Znalezienie konturów
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#największy kontur <- ramka obrazka
largest_contour = max(contours, key=cv2.contourArea)
(x, y), radius = cv2.minEnclosingCircle(largest_contour)
center = (int(x), int(y))
radius = int(radius *0.85)

circle_mask = np.zeros_like(gray)
cv2.circle(circle_mask, center, radius, 255, thickness=-1)

result = cv2.bitwise_and(image, image, mask=circle_mask)
cv2.imwrite("test1.bmp", result)

True

In [61]:
input_path = "archive\\Ulcer\\Ulcer_2024-08-07-08-31-02_23691.bmp"
#input_path = "Ulcer_2024-08-07-08-25-26_23438.bmp"

image = cv2.imread(input_path)

border_size = 50

# Dodanie czarnej ramki
padded_image = cv2.copyMakeBorder(
    image,
    border_size, border_size,  # Góra, dół
    border_size, border_size,  # Lewa, prawa
    cv2.BORDER_CONSTANT,
    value=(0, 0, 0)  # Kolor czarny
)

gray = cv2.cvtColor(padded_image, cv2.COLOR_BGR2GRAY)

# Progowanie obrazu (testuj różne wartości progów!)
_, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

# Znalezienie konturów
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
largest_contour = max(contours, key=cv2.contourArea)
# Kopia obrazu do podglądu konturów
debug_img = padded_image.copy()
cv2.drawContours(debug_img, largest_contour, -1, (0, 255, 0), 2)  # Rysuje kontury na zielono

# Wyświetlenie konturów do debugowania
cv2.imshow("Kontury", debug_img)
cv2.waitKey(0)

-1

## Usuwanie elips

In [9]:
# Konwersja do skali szarości
gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# Progowanie obrazu
_, mask = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY_INV)

# Inpainting
output = cv2.inpaint(result, mask, inpaintRadius=5, flags=cv2.INPAINT_TELEA)



In [10]:
cv2.imwrite("test.bmp", output)

True

In [ ]:
new_root = "output1" 
original_dir, filename = os.path.split(input_path) 
_, second_folder = os.path.split(original_dir)

new_dir = os.path.join(new_root, second_folder)  
os.makedirs(new_dir, exist_ok=True) 

base, ext = os.path.splitext(filename)
new_filename = f"{base}0{ext}" 

output_path = os.path.join(new_dir, new_filename)

cv2.imwrite(output_path, output)

print(f"Zapisano plik jako: {output_path}")

Zapisano plik jako: output1\AVM\AVM_2024-08-07-08-01-08_158750.bmp


## Połączenie dla wszystkich zdjęć z folderów

In [83]:

# Ścieżka do folderu z oryginalnymi obrazami
input_root = "archive"
output_root = "output2"

h, w = (512, 512)
margin_x = int(w * 0.064)
margin_y = int(h * 0.064)

# Przechodzimy rekurencyjnie przez wszystkie pliki w folderze archive
for root, _, files in os.walk(input_root):
    for file in files:
        if file.endswith(".bmp"):  # Obsługujemy tylko pliki BMP
            input_path = os.path.join(root, file)

            # Odtworzenie struktury katalogów w output1
            relative_path = os.path.relpath(root, input_root)  # np. "AVM"
            output_dir = os.path.join(output_root, relative_path)
            os.makedirs(output_dir, exist_ok=True)  # Tworzymy katalog, jeśli nie istnieje

            # Nowa nazwa pliku (dodajemy "0" przed rozszerzeniem)
            base, ext = os.path.splitext(file)
            new_filename = f"{base}{ext}"
            output_path = os.path.join(output_dir, new_filename)

            # Wczytaj obraz
            image = cv2.imread(input_path)

            # ------------------ Przetwarzanie obrazu ------------------

            # Wykrywanie czarnego tła i znajdowanie największego konturu (okrąg)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            mask = np.where(gray > 10, 255, 0).astype(np.uint8)
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            largest_contour = max(contours, key=cv2.contourArea)
            (x, y), radius = cv2.minEnclosingCircle(largest_contour)
            center = (int(x), int(y))
            radius = int(radius * 1.05)

            # Tworzenie maski kołowej i wycinanie obrazu
            circle_mask = np.zeros_like(gray)
            cv2.circle(circle_mask, center, radius, 255, thickness=-1)
            result = cv2.bitwise_and(image, image, mask=circle_mask)

            # Usuwanie czarnych obszarów
            gray_result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(gray_result, 10, 255, cv2.THRESH_BINARY_INV)
            tmp_output = cv2.inpaint(result, mask, inpaintRadius=5, flags=cv2.INPAINT_TELEA)
            
            gray = cv2.cvtColor(tmp_output, cv2.COLOR_BGR2GRAY)

            # 🔹 Tworzenie maski ignorującej rogi
            mask = np.where(gray > 10, 255, 0).astype(np.uint8)

            # 🔹 Usunięcie rogów (np. 10% od krawędzi)
            mask[:margin_y, :] = 0  # Górny pasek
            mask[-margin_y:, :] = 0  # Dolny pasek
            mask[:, :margin_x] = 0  # Lewy pasek
            mask[:, -margin_x:] = 0  # Prawy pasek

            # Znalezienie konturów w zmodyfikowanej masce
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Wybór największego konturu (bez rogów)
            largest_contour = max(contours, key=cv2.contourArea)
            
            # Znalezienie najmniejszego okręgu obejmującego kontur
            (x, y), radius = cv2.minEnclosingCircle(largest_contour)
            center = (int(x), int(y))
            radius = int(radius * 0.85)  # 🔹 Opcjonalne lekkie powiększenie

            # Tworzenie maski kołowej
            circle_mask = np.zeros_like(gray)
            cv2.circle(circle_mask, center, radius, 255, thickness=-1)

            # Wycięcie okręgu
            output = cv2.bitwise_and(tmp_output, tmp_output, mask=circle_mask)


            # ------------------ Zapisanie wyniku ------------------
            cv2.imwrite(output_path, output)


## Alternatywa dla szukania obramowania do folderu ulcer
zmiana promienia?-> działa jeśli zmiana nie jest przy krawędzi inaczej przycina zmianę

In [ ]:
padded_image = cv2.imread("output2\\Ulcer\\Ulcer_2024-08-07-08-25-38_56867.bmp")


# "output2\Ulcer\Ulcer_2024-08-07-08-25-26_73172.bmp"

# border_size = 50

# # Dodanie czarnej ramki
# padded_image = cv2.copyMakeBorder(
#     image,
#     border_size, border_size,  # Góra, dół
#     border_size, border_size,  # Lewa, prawa
#     cv2.BORDER_CONSTANT,
#     value=(0, 0, 0)  # Kolor czarny
# )

h, w = 512
margin_x = int(w * 0.058)
margin_y = int(h * 0.058)
# Konwersja do skali szarości
gray = cv2.cvtColor(padded_image, cv2.COLOR_BGR2GRAY)


#problem polega na tym ze teraz rogi nie są czarne
# 🔹 Tworzenie maski ignorującej rogi
mask = np.where(gray > 10, 255, 0).astype(np.uint8)


# 🔹 Usunięcie rogów (np. 10% od krawędzi)

mask[:margin_y, :] = 0  # Górny pasek
mask[-margin_y:, :] = 0  # Dolny pasek
mask[:, :margin_x] = 0  # Lewy pasek
mask[:, -margin_x:] = 0  # Prawy pasek

# Znalezienie konturów w zmodyfikowanej masce
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if contours:
    # Wybór największego konturu (bez rogów)
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Znalezienie najmniejszego okręgu obejmującego kontur
    (x, y), radius = cv2.minEnclosingCircle(largest_contour)
    center = (int(x), int(y))
    radius = int(radius * 0.85)  # 🔹 Opcjonalne lekkie powiększenie

    # Tworzenie maski kołowej
    circle_mask = np.zeros_like(gray)
    cv2.circle(circle_mask, center, radius, 255, thickness=-1)

    # Wycięcie okręgu
    result = cv2.bitwise_and(padded_image, padded_image, mask=circle_mask)
    
    # result = result[50:h-50,50:w-50]

    # 🔹 Dodatkowo: rysowanie wykrytego okręgu na oryginalnym obrazie
   

    # Zapis wyników
    cv2.imwrite("test1.bmp", result) 

In [50]:
image = cv2.imread("output2\\Ulcer\\Ulcer_2024-08-07-08-25-26_73172.bmp")

# border_size = 50

# # Dodanie czarnej ramki
# padded_image = cv2.copyMakeBorder(
#     image,
#     border_size, border_size,  # Góra, dół
#     border_size, border_size,  # Lewa, prawa
#     cv2.BORDER_CONSTANT,
#     value=(0, 0, 0)  # Kolor czarny
# )


gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
mask = np.where(gray > 10, 255, 0).astype(np.uint8)
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
largest_contour = max(contours, key=cv2.contourArea)
(x, y), radius = cv2.minEnclosingCircle(largest_contour)
center = (int(x), int(y))
radius = int(radius * 1.05)

# Tworzenie maski kołowej i wycinanie obrazu
circle_mask = np.zeros_like(gray)
cv2.circle(circle_mask, center, radius, 255, thickness=-1)
result = cv2.bitwise_and(image, image, mask=circle_mask)

# Usuwanie czarnych obszarów
gray_result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(gray_result, 10, 255, cv2.THRESH_BINARY_INV)
output = cv2.inpaint(result, mask, inpaintRadius=5, flags=cv2.INPAINT_TELEA)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# ------------------ Zapisanie wyniku ------------------
cv2.imwrite("test2.bmp", result) 

True